# COVID-19 Decision Stockpile Analysis
To use the surge model, change the slider for stockpile days and the table below will change to indicate minimum inventory required.

Use caution when interpreting these numbers

In [1]:
import pandas as pd
import ipywidgets as widgets
import sys

sys.path.append('../src')
from restart import NoteCompose
from output import Output
from ipywidgets import Layout

from util import set_config

import ipysheet
import numpy as np

import altair as alt

def display_sheet(sheet):
    df = to_df(sheet)
    index_name = df.index.name
    headers = list(df.index)
    df.insert(loc=0, column=index_name, value=headers)
    sheet = ipysheet.pandas_loader.from_dataframe(df)
    sheet.layout = Layout(max_height='300px', overflow_y='scroll')
    sheet.row_headers = False
    return sheet

def to_sheet(df):
    return ipysheet.pandas_loader.from_dataframe(df)

def to_df(sheet):
    return ipysheet.pandas_loader.to_dataframe(sheet)

def read_only(sheet):
    for cell in sheet.cells:
        setattr(cell, 'read_only', True)

def to_config(sheet):
    new_config['Data']['Demand m']['Level to Resource mn'] = np.array(to_df(sheet)).tolist()
    return new_config
    
def on_change(change): 
    new_sheet = sheet
    
def visualize_data(df, x=None, y=None, spacing=20):
    """Visualize data.

    Charting for the dashboard
    """
    df.index.name = "Label" if None else df.index.name
    # first reset the index to get it to be a column
    # using melt to get column form
    df_reset = df.reset_index()
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html
    # if value_vars are not specified then unpivote everything
    # Note Units is used already, so need a different name
    df_melt = df_reset.melt(id_vars=df.index.name, value_name="Count")

    # for single values
    # http://scrapingauthority.com/pandas-dataframe-filtering/
    # https://realnitinworks.netlify.app/check-object-iterability.html?utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com

    # TODO: implement filtering
    # df_filter_x = df_melt[df_melt[df.index.name].isin(x)]
    # st.dataframe(df_filter_x)
    # try:
    #  iter(y_axis)
    # df_filter_xy = df_filter_x[df_filter_x[df.columns.name].isin(y_axis)]
    # except TypeError:
    # df_filter_xy = df_filter_x[df_filter_x[df.columns.name] == y_axis]
    # df_filter_xy = df_filter_x[df_filter_x[df.columns.name].isin(y)]
    # st.dataframe(df_filter_xy)

    # Since this was published, there are more parameters for interactive
    # https://towardsdatascience.com/quickly-build-and-deploy-an-application-with-streamlit-988ca08c7e83
    # https://towardsdatascience.com/interactive-election-visualisations-with-altair-85c4c3a306f9
    # https://altair-viz.github.io/user_guide/encoding.html
    mapping = [
        ["x", alt.X],
        ["column", alt.Column],
        ["color", alt.Color],
        ["size", alt.Size],
    ]
    # make the Y axis the last column which in narrow form is always
    # the data
    y = df_melt.columns[-1]
    y_alt = y + ":Q"
    encoding = {"y": alt.Y(y_alt)}

    for col in range(0, df_melt.shape[1] - 1):
        var = df_melt.columns[col]
        # tricky, but add to the encoding dictionary
        # using the first item which is a string
        # Then the second is the function in Altair
        # to be called with which creates the right object
        title = var.rsplit(" ", 1)[0]
        # rsplit cuts off last word
        # don't render title for x axis
        if col == 0:
            encoding[mapping[col][0]] = mapping[col][1](
                var + ":N", title=None
            )
            # title=None makes Pop Level l invisible
            encoding[mapping[col + 2][0]] = mapping[col][1](var + ":N", title=title)
        # increase spacing & cut off var name for columns
        elif col == 1:
            encoding[mapping[col][0]] = mapping[col][1](
                var + ":N", spacing=spacing, title=title
            )
            # encoding[mapping[col + 1][0]] = mapping[col][1](var + ":N", title=title)
        # default configuration for color and size
        else:
            encoding[mapping[col + 1][0]] = mapping[col][1](var + ":N")
    return alt.Chart(df_melt, encoding=encoding).mark_bar()

## Bootstrap the Model
Select the parameters here to initialize the model (not implemented)

In [2]:
config = set_config('../src')
restart = NoteCompose(configdir='../src', population='oes', state='California', demand='washington')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

## Inventory Calculations

In [3]:
def set_stock(days):
    resource.demand(resource.inventory_ln_df)
    resource.set_inv_min(demand.level_total_demand_ln_df, days)
    stockpile = to_sheet(resource.inventory_ln_df)
    sheet = display_sheet(stockpile)
    display(sheet)
    display(visualize_data(resource.inventory_ln_df, spacing=30))
widgets.interact(set_stock, days=30, continuous_update=False)

interactive(children=(IntSlider(value=30, description='days', max=90, min=-30), Output()), _dom_classes=('widg…

<function __main__.set_stock(days)>

## Editable Burn Rates
These are burn rates per day for each resource, based on Washington protection level.

In [7]:
def update_burn_rate_chart(a):
    edited_df = to_df(burn)
    edited_df.columns = burn_df.columns
    edited_df.index = burn_df.index
    chart_out.clear_output(wait=True)
    with chart_out:
        display(visualize_data(edited_df, spacing = 15))
burn_df = demand.demand_per_unit_map_dn_um.df
burn = to_sheet(burn_df)
burn_widget = widgets.VBox([burn])
button_widget = widgets.Button(description="Update Chart")
button_widget.on_click(update_burn_rate_chart)
chart_out = widgets.Output()

display(burn_widget)
display(button_widget)
display(chart_out)

update_burn_rate_chart(None)

Button(description='Update Chart', style=ButtonStyle())

Output()

## Population Details
Right now the only population attribute is size, but there will be more.

In [5]:
pop_df = population.detail_pd_df
pop = to_sheet(pop_df)
read_only(pop)

display_pop = display_sheet(pop)
display(widgets.VBox([display_pop]))
display(visualize_data(pop_df))

alt.Chart(...)

## Population Levels
Protection levels for the given population.

In [6]:
level_df = population.level_pm_df

levels = to_sheet(level_df)
levels.stretch_headers = 'none'
read_only(levels)

display(widgets.VBox([display_sheet(levels)]))
# display(visualize_data(level_df)) # too many rows

# Population Demand per Resource
The daily burn rates of all resources for each subpopulation

In [7]:
demand_pn_df = demand.demand_pn_df
demand_sheet = to_sheet(demand_pn_df)
read_only(demand_sheet)

display(widgets.VBox([display_sheet(demand_sheet)]))
# visualize_data(demand_pn_df) # too many rows

## Experimenting with Vuetify

In [8]:
# import ipyvuetify as v

# vslider = v.Slider(value=30, thumb_label=True, min=0, max=120)
# vslider

# v.Container(children = [
#     display_sheet(demand_sheet)
# ])

In [ ]:
import ipywidgets as widgets
sheet = ipysheet.sheet(rows=3, columns=2, column_headers=False, row_headers=False)
cell_a = ipysheet.cell(0, 1, 1, label_left='a')
cell_b = ipysheet.cell(1, 1, 2, label_left='b')
cell_sum = ipysheet.cell(2, 1, 3, label_left='sum', read_only=True)

# create a slider linked to cell a
slider = widgets.FloatSlider(min=-10, max=10, description='a')
widgets.jslink((cell_a, 'value'), (slider, 'value'))

# changes in a or b should trigger this function
def calculate(change):
    cell_sum.value = cell_a.value + cell_b.value

sheet.cells.observe(calculate, 'value')


widgets.VBox([sheet, slider])